In [11]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from time import *
import os 
import pickle
import pandas as pd

In [12]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False  # image is unwritable
    results = model.process(image)  # make prediction
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results


def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)

In [3]:
path = ""
print(path)
frames = []
# run all directories in data
for element in os.listdir(path):
# run for all files in each subdirectory in data
    for file in os.listdir(os.path.join(path,element)):
        # don't change this
        mp_holistic = mp.solutions.holistic #holistic model
        mp_drawing = mp.solutions.drawing_utils #drawing utility
        cap = cv2.VideoCapture(os.path.join(path,element,file))
        
        with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
            while cap.isOpened():
                ref, frame = cap.read()
                if ref is False:
                    break
            # Make Detections
                image, results = mediapipe_detection(frame, holistic)
                rh = np.array([[res.x, res.y, res.z] for res in
                            results.right_hand_landmarks.landmark]) if results.right_hand_landmarks else np.full([21, 3], 0)
                lh = np.array([[res.x, res.y, res.z] for res in
                            results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.full([21, 3], 0)
                face = np.array([[res.x, res.y, res.z] for res in
                                results.face_landmarks.landmark]) if results.face_landmarks else np.full([468,3], 0)
                pose = np.array([[res.x, res.y, res.z] for res in
                                results.pose_landmarks.landmark]) if results.pose_landmarks else np.full([33,4], 0)
                # Draw Landmarks
                
                # Put all the landmarks into a 1d numpy array with the xyz coordinates for face, pose, rh and lh
                landmarks_split = [rh, lh, face, pose]
                frames.append(landmarks_split)
            
                draw_landmarks(image, results)
                
                cv2.imshow("OpenCV Capture", image)
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
        cap.release()
        cv2.destroyAllWindows()

    np.save(f"{os.path.basename(element)}.npy", np.array(frames, dtype=object), allow_pickle=True)
    

D:\Documents\Research\ASL_2024\data\DataCreation\[TEST] Collected


In [18]:
path = "./Numpy_Files"
for element in os.listdir(path):
    for file in os.listdir(os.path.join(path,element)):
        np.load(os.path.join(path,element,file), allow_pickle=True)


In [19]:
np.load("above1.npy")

FileNotFoundError: [Errno 2] No such file or directory: 'above1.npy'